In [1]:
!git clone https://github.com/SAI990323/BIGRec.git


Cloning into 'BIGRec'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 116 (delta 57), reused 77 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (116/116), 1.97 MiB | 4.10 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [2]:
%cd BIGRec

/content/BIGRec


In [3]:
!pip install -r requirements.txt


  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-t87f__85
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-t87f__85
  Resolved https://github.com/huggingface/peft.git to commit 2f063e6342235842a00bca150a98854b89117986
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Cannot install peft 0.14.1.dev0 (from git+https://github.com/huggingface/peft.git) and peft==0.3.0 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested peft==0.3.0
    The user requested peft 0.14.1.dev0 (from git+https://github.com/huggingface/peft.git)

To fix this you could try to:
1. lo

In [4]:
import os
os.getcwd()

'/content/BIGRec'

In [5]:
%cd data/movie/

/content/BIGRec/data/movie


In [7]:
f = open('ratings.dat', 'r')
data = f.readlines()
f = open('movies.dat', 'r', encoding='ISO-8859-1')
movies = f.readlines()
movie_names = [_.split('::')[1] for _ in movies]
movie_ids = [_.split('::')[0] for _ in movies]
movie_dict = dict(zip(movie_ids, movie_names))
id_mapping = dict(zip(movie_ids, range(len(movie_ids))))

In [8]:
interaction_dicts = dict()
for line in data:
    user_id, movie_id, rating, timestamp = line.split('::')
    if user_id not in interaction_dicts:
        interaction_dicts[user_id] = {
            'movie_id': [],
            'rating': [],
            'timestamp': [],
            'movie_title': [],
        }
    interaction_dicts[user_id]['movie_id'].append(movie_id)
    interaction_dicts[user_id]['rating'].append(int(float(rating) > 3.0))
    interaction_dicts[user_id]['timestamp'].append(timestamp)
    interaction_dicts[user_id]['movie_title'].append(movie_dict[movie_id])


In [9]:
with open('all.csv', 'w') as f:
    import csv
    writer = csv.writer(f)
    writer.writerow(['user_id', 'item_id', 'rating', 'timestamp', 'item_title'])
    for user_id, user_dict in interaction_dicts.items():
        writer.writerow([user_id, user_dict['movie_id'], user_dict['rating'], user_dict['timestamp'], user_dict['movie_title']])


In [10]:
sequential_interaction_list = []
seq_len = 10
for user_id in interaction_dicts:
    temp = zip(interaction_dicts[user_id]['movie_id'], interaction_dicts[user_id]['rating'], interaction_dicts[user_id]['timestamp'], interaction_dicts[user_id]['movie_title'])
    temp = sorted(temp, key=lambda x: int(x[2]))
    result = zip(*temp)
    interaction_dicts[user_id]['movie_id'], interaction_dicts[user_id]['rating'], interaction_dicts[user_id]['timestamp'], interaction_dicts[user_id]['movie_title'] = [list(_) for _ in result]
    for i in range(10, len(interaction_dicts[user_id]['movie_id'])):
        sequential_interaction_list.append(
            [user_id, interaction_dicts[user_id]['movie_title'][i - seq_len: i],interaction_dicts[user_id]['movie_id'][i-seq_len:i], interaction_dicts[user_id]['rating'][i-seq_len:i], interaction_dicts[user_id]['movie_id'][i], interaction_dicts[user_id]['rating'][i], interaction_dicts[user_id]['timestamp'][i].strip('\n')]
        )
print(len(sequential_interaction_list))

939809


In [11]:
import csv
sequential_interaction_list = sorted(sequential_interaction_list, key=lambda x: int(x[-1]))
with open('./train.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id', 'history_movie_title', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'])
    writer.writerows(sequential_interaction_list[:int(len(sequential_interaction_list)*0.8)])
with open('./valid.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id', 'history_movie_title', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'])
    writer.writerows(sequential_interaction_list[int(len(sequential_interaction_list)*0.8):int(len(sequential_interaction_list)*0.9)])
with open('./test.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id', 'history_movie_title', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'])
    writer.writerows(sequential_interaction_list[int(len(sequential_interaction_list)*0.9):])

In [12]:
import json
import pandas as pd
import random
import numpy as np
def csv_to_json(input_path, output_path, sample=False):
    data = pd.read_csv(input_path)
    if sample:
        data = data.sample(n=5000, random_state=42).reset_index(drop=True)
        data.to_csv(output_path[:-5] + ".csv", index=False)
    json_list = []
    for index, row in data.iterrows():
        row['history_movie_id'] = eval(row['history_movie_id'])
        row['history_movie_title'] = eval(row['history_movie_title'])
        L = len(row['history_movie_id'])
        history = "The user has watched the following movies before:"
        for i in range(L):
            if i == 0:
                history += "\"" + row['history_movie_title'][i] + "\""
            else:
                history += ", \"" + row['history_movie_title'][i] + "\""
        target_movie_name = "\"" + movie_dict[str(row['movie_id'])] + "\""
        json_list.append({
            "instruction": "Given a list of movies the user has watched before, please recommend a new movie that the user likes to the user.",
            "input": f"{history}\n ",
            "output": target_movie_name,
        })

    with open(output_path, 'w') as f:
        json.dump(json_list, f, indent=4)


In [13]:
csv_to_json('./train.csv', './train.json')
csv_to_json('./valid.csv', './valid.json')
csv_to_json('./test.csv', './test.json')
csv_to_json('./valid.csv', './valid_5000.json', sample=True)
csv_to_json('./test.csv', './test_5000.json', sample=True)

In [15]:
%cd ..
os.getcwd()

/content/BIGRec


'/content/BIGRec'

In [ ]:

# 设置超参数
seeds = [0, 1, 2]
learning_rates = [1e-4]
dropouts = [0.05]
samples = [1024]

# 循环运行模型
for seed in seeds:
    for lr in learning_rates:
        for dropout in dropouts:
            for sample in samples:
                print(f"lr: {lr}, dropout: {dropout}, seed: {seed},")
                # 构建命令
                command = f"CUDA_VISIBLE_DEVICES=$1 python train.py \
                    --base_model YOUR_LLAMA_PATH/ \
                    --train_data_path '[\"./data/movie/train.json\"]'   \
                    --val_data_path '[\"./data/movie/valid_5000.json\"]' \
                    --output_dir /model/movie/{seed}_{sample} \
                    --batch_size 128 \
                    --micro_batch_size 4 \
                    --num_epochs 50 \
                    --learning_rate {lr} \
                    --cutoff_len 512 \
                    --lora_r 8 \
                    --lora_alpha 16\
                    --lora_dropout {dropout} \
                    --lora_target_modules '[q_proj,v_proj]' \
                    --train_on_inputs \
                    --group_by_length \
                    --resume_from_checkpoint 'XXX' \
                    --seed {seed} \
                    --sample {sample}"
                # 执行命令
                os.system(command)


In [16]:
!pip install fire

  Using cached fire-0.7.0.tar.gz (87 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=d2a4a75eac34f692573c54f5db9547c560065db719f39032c4e0f16b403acab1
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [17]:
!pip install datasets

  Using cached datasets-3.3.2-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.0 MB/s eta 0:00:00


In [24]:
!pip install bitsandbytes>=0.43.0

In [23]:
import torch

print(torch.cuda.is_available())


True


In [26]:
!pip uninstall bitsandbytes
!pip install https://github.com/jllllll/bitsandbytes-windows-webui/releases/download/wheels/bitsandbytes-0.41.0-py3-none-win_amd64.whl

Found existing installation: bitsandbytes 0.45.3
Uninstalling bitsandbytes-0.45.3:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/bitsandbytes-0.45.3.dist-info/*
    /usr/local/lib/python3.11/dist-packages/bitsandbytes/*
Proceed (Y/n)? Y
  Successfully uninstalled bitsandbytes-0.45.3
ERROR: bitsandbytes-0.41.0-py3-none-win_amd64.whl is not a supported wheel on this platform.


In [25]:
import subprocess

def run_command(command):
    result = subprocess.run(command, shell=True, text=True, capture_output=True)
    print("Status:", result.returncode)  # 打印返回状态码
    print("Output:", result.stdout)      # 打印标准输出
    if result.stderr:                    # 如果有标准错误输出，也打印它
        print("Errors:", result.stderr)

seed = 0
lr = 1e-4
dropout = 0.05
sample = 1024

# 示例命令
command = f"CUDA_VISIBLE_DEVICES=$1 python train.py \
    --base_model 'huggyllama/llama-7b' \
    --train_data_path '[\"./data/movie/train.json\"]'   \
    --val_data_path '[\"./data/movie/valid_5000.json\"]' \
    --output_dir /model/movie/{seed}_{sample} \
    --batch_size 128 \
    --micro_batch_size 4 \
    --num_epochs 50 \
    --learning_rate {lr} \
    --cutoff_len 512 \
    --lora_r 8 \
    --lora_alpha 16\
    --lora_dropout {dropout} \
    --lora_target_modules '[q_proj,v_proj]' \
    --train_on_inputs \
    --group_by_length \
    --resume_from_checkpoint 'XXX' \
    --seed {seed} \
    --sample {sample}"
run_command(command)

Status: 1
Output: Training Alpaca-LoRA model with params:
base_model: huggyllama/llama-7b
train_data_path: ['./data/movie/train.json']
val_data_path: ['./data/movie/valid_5000.json']
sample: 1024
seed: 0
output_dir: /model/movie/0_1024
batch_size: 128
micro_batch_size: 4
num_epochs: 50
learning_rate: 0.0001
cutoff_len: 512
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['q_proj', 'v_proj']
train_on_inputs: True
group_by_length: True
wandb_project: 
wandb_run_name: 
wandb_watch: 
wandb_log_model: 
resume_from_checkpoint: XXX


Errors: 2025-03-13 17:12:25.652996: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741885945.687417    4015 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741885945.698163    4015 cuda_blas.cc:1418] 

In [27]:
!git clone https://github.com/JiangM-C/IFairLRS.git


Cloning into 'IFairLRS'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 26 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 111.69 KiB | 893.00 KiB/s, done.
Resolving deltas: 100% (8/8), done.


In [29]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [33]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `Shelly Sun` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Shelly 

In [36]:
!pip install --upgrade peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 7.4 MB/s eta 0:00:00


In [42]:
import torch
torch.cuda.empty_cache()

In [ ]:
import sys
import fire
import gradio as gr
import torch
torch.set_num_threads(1)
import transformers
import json
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
from peft import PeftModel
from transformers import GenerationConfig,  LlamaTokenizer, LlamaForCausalLM
from sklearn.metrics import roc_auc_score
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# try:
#     if torch.backends.mps.is_available():
#         device = "mps"
# except:  # noqa: E722
#     pass


def main(
    load_8bit: bool = False,
    base_model: str = 'mrzlab630/weights_Llama_7b',
    lora_weights: str = 'mrzlab630/weights_Llama_7b',
    test_data_path: str = "/content/BIGRec/data/movie/test_5000.json",
    result_json_data: str = "/content/BIGRec/data/movie/ml-1m_result.json",
    task_type: str = "task1",
    share_gradio: bool = False,
    tau: float = 1,
):
    assert (
        base_model
    ), "Please specify a --base_model, e.g. --base_model='decapoda-research/llama-7b-hf'"

    tokenizer = LlamaTokenizer.from_pretrained(base_model)
    if device == "cuda":
        model = LlamaForCausalLM.from_pretrained(
            base_model,
            load_in_8bit=load_8bit,
            torch_dtype=torch.float16,
            device_map="auto",
            offload_folder="offload",
        )

        # model = PeftModel.from_pretrained(
        #     model,
        #     lora_weights,
        #     torch_dtype=torch.float16,
        #     device_map="auto",
        # )
    elif device == "mps":
        model = PeftModel.from_pretrained(
            model,
            lora_weights,
            device_map={"": device},
            torch_dtype=torch.float16,
        )
    else:
        model = PeftModel.from_pretrained(
            model,
            lora_weights,
            device_map={"": device},
        )

    # model.set_tau(tau)



    tokenizer.padding_side = "left"


    # unwind broken decapoda-research config
    model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
    model.config.bos_token_id = 1
    model.config.eos_token_id = 2

    if not load_8bit:
        model.half()  # seems to fix bugs for some users.

    model.eval()
    if torch.__version__ >= "2" and sys.platform != "win32":
        model = torch.compile(model)

    def evaluate(
        instructions,
        inputs=None,
        temperature=0,
        top_p=0.9,
        top_k=40,
        num_beams=5,
        max_new_tokens=128,
        **kwargs,
    ):
        prompt = [generate_prompt(instruction, input) for instruction, input in zip(instructions, inputs)]
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
        generation_config = GenerationConfig(
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            num_beams=num_beams,
            num_return_sequences=num_beams,
            **kwargs,
        )
        with torch.no_grad():
            generation_output = model.generate(
                **inputs,
                generation_config=generation_config,
                return_dict_in_generate=True,
                output_scores=True,
                max_new_tokens=max_new_tokens,
                # batch_size=batch_size,
            )
        # print(generation_output.keys())
        # print(generation_output.sequences_scores.shape)
        # print(generation_output.beam_indices.shape)
        s = generation_output.sequences
        output = tokenizer.batch_decode(s, skip_special_tokens=True)
        output = [_.split('Response:\n')[-1] for _ in output]
        real_outputs = [output[i * num_beams: (i + 1) * num_beams] for i in range(len(output) // num_beams)]
        return real_outputs
        # return output.strip(' '), logits

    # testing code for readme

    logit_list = []
    gold_list= []
    outputs = []
    logits = []
    from tqdm import tqdm
    with open(test_data_path, 'r') as f:
        test_data = json.load(f)
        # for i, test in tqdm(enumerate(test_data)):
        instructions = [_['instruction'] for _ in test_data]
        inputs = [_['input'] for _ in test_data]
        def batch(list, batch_size=8):
            chunk_size = (len(list) - 1) // batch_size + 1
            for i in range(chunk_size):
                yield list[batch_size * i: batch_size * (i + 1)]
        for i, batch in tqdm(enumerate(zip(batch(instructions), batch(inputs)))):
            instructions, inputs = batch
            output = evaluate(instructions, inputs, temperature=tau)
            outputs = outputs + output

        for i, test in tqdm(enumerate(test_data)):
            test_data[i]['predict'] = outputs[i]


    with open(result_json_data, 'w') as f:
        json.dump(test_data, f, indent=4)

    # data[train_sce][test_sce][model_name][seed][sample] = roc_auc_score(gold_list, logit_list)
    # f = open(result_json_data, 'w')
    # json.dump(data, f, indent=4)
    # f.close()

def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
{instruction}

### Input:
{input}

### Response:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
{instruction}

### Response:
"""


if __name__ == "__main__":
    # fire.Fire(main)
    main()

/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1536: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2137: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


In [41]:
torch.cuda.memory_summary()

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 3         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  14399 MiB |  14750 MiB |  55974 MiB |  41574 MiB |\n|       from large pool |  14398 MiB |  14750 MiB |  55970 MiB |  41571 MiB |\n|       from small pool |      0 MiB |      1 MiB |      3 MiB |      2 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |  14399 MiB |  14750 MiB |  55974 MiB |  41574 MiB |\n|       from large pool |  14398 MiB |  14750 MiB |